In [ ]:
from pyspark.sql import SparkSession
import requests
import matplotlib.pyplot as plt
import pandas as pd
from pyspark.sql.functions import substring, col, sum, when, lit
import seaborn as sns

In [ ]:
#create sparksession entry point to apache spark allows to work with dataframes and transformations
spark = SparkSession.builder.getOrCreate()

#read json into rdd using .rdd
branch_rdd = spark.read.json("CDW_SAPP_BRANCH.JSON").rdd
credit_rdd = spark.read.json("CDW_SAPP_CREDIT.JSON").rdd
customer_rdd = spark.read.json("CDW_SAPP_CUSTMER.JSON").rdd

In [ ]:
#map applies given function lambda which applies transformations using row
#row represents every element of the rdd 
branch_transformed_rdd = branch_rdd.map(lambda row: (
    #direct moves, no transformations needed
    int(row["BRANCH_CODE"]),
    str(row["BRANCH_NAME"]),
    str(row["BRANCH_STREET"]),
    str(row["BRANCH_CITY"]),
    str(row["BRANCH_STATE"]),
    
    #takes values from BRANCH_ZIP and if null will set row to 99999
    int(row["BRANCH_ZIP"]) if row["BRANCH_ZIP"] is not None else 99999,
    #transforms BRANCH_PHONE to phone number format
    "(" + row["BRANCH_PHONE"][:3] + ")" + row["BRANCH_PHONE"][3:6] + "-" + row["BRANCH_PHONE"][6:],
))      

#transformed rdd is converted dataframe using .toDF
branch_transformed_df = branch_transformed_rdd.toDF(["BRANCH_CODE", "BRANCH_NAME", "BRANCH_STREET", "BRANCH_CITY", "BRANCH_STATE", "BRANCH_ZIP", "BRANCH_PHONE"])

#output the new dataframe
branch_transformed_df.show()

In [ ]:
credit_transformed_rdd = credit_rdd.map(lambda row: (
    str(row["CREDIT_CARD_NO"]),

    #concatenate year month day into TIMEID YYYYMMDD format using zfill on single digit "MONTH" "DAY"
    str(row["YEAR"]) + str(row["MONTH"]).zfill(2) + str(row["DAY"]).zfill(2),

    #direct move no transformations needed
    int(row["CUST_SSN"]),
    int(row["BRANCH_CODE"]),
    str(row["TRANSACTION_TYPE"]),
    float(row["TRANSACTION_VALUE"]),
    int(row["TRANSACTION_ID"])
))


credit_transformed_df = credit_transformed_rdd.toDF(["CREDIT_CARD_NO", "TIMEID", "CUST_SSN", "BRANCH_CODE", "TRANSACTION_TYPE", "TRANSACTION_VALUE", "TRANSACTION_ID"])

credit_transformed_df.show()

In [ ]:
customer_transformed_rdd = customer_rdd.map(lambda row: (
    int(row["SSN"]),
    # .title() makes it title case
    str(row["FIRST_NAME"]).title(),
    # .lower() makes it lower case
    str(row["MIDDLE_NAME"]).lower(),
    str(row["LAST_NAME"]).title(),
    str(row["CREDIT_CARD_NO"]),
    #concatenate STREET_NAME and APT_NO using + operator
    str(row["STREET_NAME"]) + ", " + str(row["APT_NO"]),
    str(row["CUST_CITY"]),
    str(row["CUST_STATE"]),
    str(row["CUST_COUNTRY"]),
    int(row["CUST_ZIP"]),
    #turn into phone number format, this time phone number is 7 digits +XXX to make it 10 to fit the format
    "(" + str(row["CUST_PHONE"])[:3] + ")" + str(row["CUST_PHONE"])[3:6] + "-" + str(row["CUST_PHONE"])[6:] + "XXX",
    str(row["CUST_EMAIL"]),
    row["LAST_UPDATED"]
))

customer_transformed_df = customer_transformed_rdd.toDF(["SSN", "FIRST_NAME", "MIDDLE_NAME", "LAST_NAME", "CREDIT_CARD_NO","FULL_STREET_ADDRESS", "CUST_CITY", "CUST_STATE", "CUST_COUNTRY","CUST_ZIP", "CUST_PHONE", "CUST_EMAIL", "LAST_UPDATED"])

customer_transformed_df.show()

In [ ]:
#write transformed branch into mysql using jdbc
branch_transformed_df.write.format("jdbc").options(
    url="jdbc:mysql://localhost/creditcard_capstone",
    driver="com.mysql.jdbc.Driver",
    dbtable="CDW_SAPP_BRANCH",
    user="root",
    password="password"
#use overwrite to prevent redundant data
).mode("overwrite").save()

#read the new table from sql
branch_sql = spark.read.format("jdbc").options(
    url="jdbc:mysql://localhost/creditcard_capstone",
    driver="com.mysql.jdbc.Driver",
    dbtable="CDW_SAPP_BRANCH",
    user="root",
    password="password"
#stored contents into branch_sql
).load()

#display sql table contents
branch_sql.show()

In [ ]:
credit_transformed_df.write.format("jdbc").options(
    url="jdbc:mysql://localhost/creditcard_capstone",
    driver="com.mysql.jdbc.Driver",
    dbtable="CDW_SAPP_CREDIT_CARD",
    user="root",
    password="password"
).mode("overwrite").save()

credit_sql = spark.read.format("jdbc").options(
    url="jdbc:mysql://localhost/creditcard_capstone",
    driver="com.mysql.jdbc.Driver",
    dbtable="CDW_SAPP_CREDIT_CARD",
    user="root",
    password="password"
).load()

credit_sql.show()

In [ ]:
customer_transformed_df.write.format("jdbc").options(
    url="jdbc:mysql://localhost/creditcard_capstone",
    driver="com.mysql.jdbc.Driver",
    dbtable="CDW_SAPP_CUSTOMER",
    user="root",
    password="password"
).mode("overwrite").save()

customer_sql = spark.read.format("jdbc").options(
    url="jdbc:mysql://localhost/creditcard_capstone",
    driver="com.mysql.jdbc.Driver",
    dbtable="CDW_SAPP_CUSTOMER",
    user="root",
    password="password"
).load()

customer_sql.show()

In [ ]:
def transactions_by_zip_month_year(credit_sql, customer_sql):
    while True:
        #user input zipcode then validated
        zip_code = input("Please enter your zip code. ")
       
        if (len(zip_code) == 5 or len(zip_code) == 4) and zip_code.isdigit():
            break
        else:           
            print("Invalid zip code, please enter your zip code.")

    while True:
        #user input month then validated
        month = input("Please enter the month of the transactions in numeric form: ")
        
        if month.isdigit() and 1 <= int(month) <= 12:
            month = month.zfill(2)
            break
        else:
            print("Invalid month. Please enter a number from 1 to 12 matching your desired month.")

    #parse sliced TIMEID for unique years
    unique_years = [str(row.TIMEID)[0:4] for row in credit_sql.select("TIMEID").distinct().collect()]   

    #Validate user input for the year by accepting inputs that match TIMEID slice
    while True:
        year = input("Please enter the year of your transactions: ")
        if year in unique_years:
            break
        else:
            print("Invalid input. Please enter the year of the transaction.")

    #inner join credit_sql and credit_sql by common column SSN 
    custcred_df = credit_sql.join(customer_sql, credit_sql.CUST_SSN == customer_sql.SSN, 'inner').orderBy(col("CUST_SSN").desc())

    #goes through crustcred_df matching user inputted zip_code month year against CUST_ZIP column and slices of TIMEID
    transactions = custcred_df.filter(
        (custcred_df.CUST_ZIP == zip_code) &
        (substring(custcred_df.TIMEID, 1, 4) == year) &
        (substring(custcred_df.TIMEID, 5, 2) == month)
    ).withColumn("day", substring(custcred_df.TIMEID, 7, 2)).orderBy(col("day").desc())

   
    if transactions.count() == 0:
        print("That information does not match our records.")
    else:
        transactions.show()

transactions_by_zip_month_year(credit_sql, customer_sql)

In [ ]:
def transactions_by_type(credit_sql):
    #retrieve all unique rows of TRANSACTION_TYPE and store it in a list to match against user inputs
    unique_transaction_types = credit_sql.select("TRANSACTION_TYPE").distinct().orderBy("TRANSACTION_TYPE")
    transaction_types_list = [row.TRANSACTION_TYPE.lower() for row in unique_transaction_types.collect()]

    #validates user input against transaction_type list
    while True:
        transaction_type = input("Please enter the transaction type: ").lower()
        if transaction_type in transaction_types_list:
            break
        else:
            print("Invalid transaction type. Please enter a valid option.")

    #goes through credit_sql finding every instance users input matches
    transaction_types = credit_sql.filter(col("TRANSACTION_TYPE").like(transaction_type))

    if transaction_types.count() == 0:
        print("No transactions found for the given type.")
    else:
        total_transactions = transaction_types.count()
        total_value = transaction_types.select(col("TRANSACTION_VALUE")).groupBy().sum().collect()[0][0]

        print("Total transactions of " + transaction_type.title() + ": " + str(total_transactions))
        print("Total transaction value of " + transaction_type.title() + ": " + str(total_value))

transactions_by_type(credit_sql)

In [ ]:
def transactions_by_state(branch_sql):
    #retrieve all unique rows of BRANCH_STATE and store it in a list to match against user inputs
    unique_states = branch_sql.select("BRANCH_STATE").distinct().orderBy("BRANCH_STATE")
    states_list = [row.BRANCH_STATE.lower() for row in unique_states.collect()]
    #validate user input for state
    while True:
        state = input("Please enter the state of the transactions: ").lower()
        if state in states_list:
            break
        else:
            print("Invalid state. Please enter a valid state code.")

    #inner join credit_sql and branch_sql by common column BRANCH_CODE to use matching data
    branchcred_df = credit_sql.join(branch_sql, credit_sql.BRANCH_CODE == branch_sql.BRANCH_CODE, 'inner')
    
    #goes through credit_sql finding every instance users input matches data
    transactions_state = branchcred_df.filter(branch_sql.BRANCH_STATE == state.upper())

    if transactions_state.count() == 0:
        print("No transactions found for branches in the given state.")
    else:
        total_transactions = transactions_state.count()
        total_value = transactions_state.select(sum("TRANSACTION_VALUE")).collect()[0][0]

        print("Number of transactions for branches in  " + state.upper() + ": " + str(total_transactions))
        print("Total transaction value for branches in  " + state.upper() + ": " + str(total_value))

transactions_by_state(branch_sql)

In [ ]:
def customer_account_details(customer_sql):
    #asks for user input and validates that the input is 4 digits and a number
    while True:
        ssn = input("Please enter the last 4 digits of your SSN: ")
        if ssn.isdigit() and len(ssn) == 4:
            break
        else:
            print("Invalid input. Please enter the last 4 digits of your SSN.")

    #filtering by column "SSN" in customer_sql for last 4 digits and matching it to user input
    account_details = customer_sql.filter(customer_sql.SSN.substr(-4, 4) == ssn)

    if account_details.count() == 0:
        print("No account details found for the given SSN.")
    else:
        account_details.show()

customer_account_details(customer_sql)

In [ ]:
def modify_account_details(customer_sql):

    while True:
        #user input and validation of ssn
        ssn_last_4 = input("Please enter the last 4 digits of your SSN (or 'q' to exit at anytime): ")
        if ssn_last_4.lower() == "q":
            return
        elif ssn_last_4.isdigit() and len(ssn_last_4) == 4:
            break
        else:
            print("Invalid input. Please enter the last 4 digits of your SSN.")

    #filtered_customer stores values from each row of the customer with the inputted ssn
    filtered_customer = customer_sql.filter(customer_sql.SSN.substr(-4, 4) == ssn_last_4)

    filtered_customer.show()

    if filtered_customer.count() == 0:
        print("No account found for the provided SSN.")
        return
    #asks user if they want to modify code or exit
    while True:
        choice = input("Do you wish to make any changes? (y/n): ")
        if choice.lower() == "q":
            return
        elif choice.lower() == "n":
            return
        elif choice.lower() == "y":
            break
        else:
            print("Invalid choice. Please enter 'y' to continue or 'n' to exit.")

    account_details_columns = filtered_customer.columns

    #removes ssn and last_updated from options user can modify
    account_details_columns.remove("SSN")
    account_details_columns.remove("LAST_UPDATED")
    #use enumerate to create a numbered list using for loop to iterate through each value of account_detail_column
    for i, account_detail_column in enumerate(account_details_columns, start=1):
        print(str(i) + ". " + account_detail_column)

    #user selects from the enumerated options
    selected_option = input("Select the number corresponding to the account detail you want to modify: ")

    if selected_option.lower() == "q":
        return
    #validates input
    while True:
        if selected_option.isdigit() and 1 <= int(selected_option) <= len(account_details_columns):
            break
        else:
            print("Invalid option. Please enter a valid number.")
            selected_option = input("Select the number corresponding to the account detail you want to modify: ")

    if selected_option.lower() == "q":
            return

    selected_column = account_details_columns[int(selected_option)-1]

    #input modification
    new_value = input("Enter new value for " + selected_column)

    if new_value.lower() == "q":
        return

    #validates that the user is entering a value
    while len(new_value) == 0:
        print("Invalid input. Please enter a value.")
        new_value = input("Enter new value for " + selected_column)

        if new_value.lower() == "q":
            return

    #use lit to replace value in selected column with user input
    filtered_customer = filtered_customer.withColumn(selected_column, lit(new_value))

    #displays updated information and asks user if it is correct
    while True:
        filtered_customer.show()
        confirm = input("Is the information correct? (y/n): ")
        if confirm.lower() == "q":
            return
        elif confirm.lower() == "y":
            break
        elif confirm.lower() == "n":
                return
        else:
            print("Invalid choice. Please enter 'y' to exit, 'n' to start over.")

    for column in account_details_columns:
        if column == selected_column:
            customer_sql = customer_sql.withColumn(column, when(customer_sql.SSN.substr(-4, 4) == ssn_last_4, lit(new_value)).otherwise(customer_sql[column]))

    print("Updated Account Details:")
    customer_sql.filter(customer_sql.SSN.substr(-4, 4) == ssn_last_4).show()

    customer_sql.write.format("jdbc").options(
        url="jdbc:mysql://localhost/creditcard_capstone",
        driver="com.mysql.jdbc.Driver",
        dbtable="CDW_SAPP_CUSTOMER",
        user="root",
        password="password"
    ).mode("overwrite").save()

modify_account_details(customer_sql)

In [ ]:
def calculate_transaction_sums(credit_sql):
    while True:
        ssn_last_4 = input("Please enter the last 4 digits of your SSN (or 'q' to exit at anytime): ")
        if ssn_last_4.lower() == "q":
            return
        elif ssn_last_4.isdigit() and len(ssn_last_4) == 4:
            break
        else:
            print("Invalid input. Please enter the last 4 digits of your SSN.")

    #gets user input and validates it
    while True:
        month = input("Please enter the month of your transactions in numeric form (1-12): ")
        if ssn_last_4.lower() == "q":
            return
        elif month.isdigit() and 1 <= int(month) <= 12:
            break
        else:
            print("Invalid input. Please enter a numeric value between 1 and 12.")
     
    #collects unique years
    unique_years = [str(row.TIMEID)[0:4] for row in credit_sql.select("TIMEID").distinct().collect()]  

    while True:
        year = input("Please enter the year of your transactions: ")
        if ssn_last_4.lower() == "q":
            return
        elif year in unique_years:
            break
        else:
            print("Invalid input. Please enter a valid year.")

    #stores rows that match the sliced CUST_SSN and sliced TIMEID
    filtered_transactions = credit_sql.filter(
        (credit_sql.CUST_SSN.substr(-4, 4) == ssn_last_4) &
        (substring(credit_sql.TIMEID, 5, 2).cast("integer") == int(month)) &
        (substring(credit_sql.TIMEID, 1, 4).cast("integer") == int(year))
    )

    #displays information regarding the transactions
    filtered_transactions.select("TRANSACTION_ID", "TRANSACTION_TYPE", "TRANSACTION_VALUE").show()

    #takes sum and .first collects the result
    total_transaction_value = filtered_transactions.agg(sum("TRANSACTION_VALUE")).first()[0]
    
    print("Total Transaction Value:", total_transaction_value)


calculate_transaction_sums(credit_sql)

In [ ]:
def display_transactions_between_dates(credit_sql):

    #verify ssn
    while True:
        ssn_last_4 = input("Please enter the last 4 digits of your SSN: ")
        if len(ssn_last_4) == 4 and ssn_last_4.isdigit():
            break
        print("Invalid input. Please enter the last 4 digits of your SSN.")
    #user input start date and end date
    while True:
        start_day = input("{Please enter the start day: ")
        start_month = input("Pleas enter the start month: ")
        start_year = input("Please enter the start year: ")

        end_day = input("Pleasenter the end day: ")
        end_month = input("Please enter the end month: ")
        end_year = input("Please enter the end year: ")
    #verifies the values are numbers and correct length and zfill for single digit month and day values
        if (start_day.isdigit() and start_month.isdigit() and start_year.isdigit() and end_day.isdigit() and end_month.isdigit() and end_year.isdigit() and
            len(start_day) <= 2 and len(start_month) <= 2 and len(start_year) == 4 and len(end_day) <= 2 and len(end_month) <= 2 and len(end_year) == 4
        ):
            start_day = start_day.zfill(2)
            start_month = start_month.zfill(2)
            end_day = end_day.zfill(2)
            end_month = end_month.zfill(2)
            break
        print("Invalid input. Please enter valid day, month, and year.")

    #takes the day month and year values and formats it into TIMEID's format through concatenation
    start_date_string = start_year + start_month + start_day
    end_date_string = end_year + end_month + end_day

    #stores rows that match ssn and are within start and end date
    filtered_transactions = credit_sql.filter(
        (credit_sql.CUST_SSN.substr(-4, 4) == ssn_last_4) &
        (credit_sql.TIMEID.substr(1, 8) >= start_date_string) &
        (credit_sql.TIMEID.substr(1, 8) <= end_date_string)
    ).orderBy(col("TIMEID").desc())

    #display relevant columns
    filtered_transactions.select("TIMEID", "TRANSACTION_ID", "TRANSACTION_TYPE", "TRANSACTION_VALUE").show()

display_transactions_between_dates(credit_sql)

In [ ]:
sns.set(style="whitegrid")

transaction_count = credit_sql.groupby("TRANSACTION_TYPE").count()
transaction_count_pd = transaction_count.toPandas()
transaction_count_pd.plot(kind="bar", x="TRANSACTION_TYPE", y="count", legend=False, color="purple", width=0.6)

plt.title("Total Count for each Transaction Type", fontsize=14)
plt.xlabel("Transaction Type", fontsize=14)
plt.ylabel("Total Count", fontsize=14)
plt.xticks(rotation=45, ha="right")
plt.tight_layout()

for i, value in enumerate(transaction_count_pd["count"]):
    plt.text(i, value, str(value), ha="center", va="bottom")

plt.show()

In [ ]:
sns.set(style="whitegrid")

state_counts = customer_sql.groupby("CUST_STATE").count()
state_counts_pd = state_counts.toPandas()
state_counts_pd.plot(kind="bar", x="CUST_STATE", y="count", color="purple", legend=False, width=0.6)


plt.title("Number of Customers by State", fontsize=14)
plt.xlabel("State", fontsize=14)
plt.ylabel("Number of Customers", fontsize=14)
plt.xticks(rotation=90)
plt.tight_layout()

plt.show()

In [ ]:
sns.set(style="whitegrid")

customer_transaction_sum = credit_sql.groupby("CUST_SSN").agg(sum("TRANSACTION_VALUE").alias("TRANSACTION_VALUE"))
customer_transaction_sum = customer_transaction_sum.toPandas()
customer_transaction_sum = customer_transaction_sum.sort_values("TRANSACTION_VALUE", ascending=False)
top_10_customers = customer_transaction_sum.head(10)
top_10_customers.plot(kind="bar", x="CUST_SSN", y="TRANSACTION_VALUE", legend=False, color="purple")

plt.title("Top Paying Customers", fontsize=14)
plt.xlabel("Customer SSN", fontsize=14)
plt.ylabel("Money Spent(Dollars)", fontsize=14)
plt.xticks(rotation=45, ha="right")
plt.tight_layout()

plt.show()

In [ ]:
api_url = "https://raw.githubusercontent.com/platformps/LoanDataset/main/loan_data.json"

response = requests.get(api_url)

status_code = response.status_code
print("Status Code:", status_code)

loandata_df = spark.read.json(spark.sparkContext.parallelize([response.text]))

loandata_df.write.format("jdbc").options(
    url="jdbc:mysql://localhost:3306/creditcard_capstone",
    driver="com.mysql.jdbc.Driver",
    dbtable="CDW_SAPP_loan_application",
    user="root",
    password="password"
).mode("overwrite").save()

loandata_sql = spark.read.format("jdbc").options(
    url="jdbc:mysql://localhost:3306/creditcard_capstone",
    driver="com.mysql.jdbc.Driver",
    dbtable="CDW_SAPP_loan_application",
    user="root",
    password="password"
).load()

loandata_sql.show()

In [ ]:
self_employed_applications = loandata_sql.filter(loandata_sql["Self_Employed"] == "Yes")
total_self_employed = self_employed_applications.count()
approved_self_employed = self_employed_applications.filter(self_employed_applications["Application_Status"] == "Y").count()
approval_percentage = (approved_self_employed / total_self_employed) * 100

plt.pie([approval_percentage, 100 - approval_percentage], labels=['Accepted', 'Rejected'], autopct='%1.1f%%', startangle = 90)
plt.title('Self-Employed Applicants')
plt.show()

In [ ]:
married_male_applicants = loandata_sql.filter((loandata_sql.Married == "Yes") & (loandata_sql.Gender == "Male"))
rejected_applications = married_male_applicants.filter(loandata_sql.Application_Status == "N").count()
total_married_applicants = married_male_applicants.count()
percentage_rejection = (rejected_applications / total_married_applicants) * 100
df = pd.DataFrame({'Percentage': [percentage_rejection, 100 - percentage_rejection]}, index=['Rejected', 'Accepted'])
df.plot(kind='pie', y='Percentage', autopct='%1.1f%%', startangle=90, legend=False)

plt.title('Married Male Applicants')
plt.ylabel('')  

plt.show()

In [ ]:
credit_sql = credit_sql.withColumn("month", substring(credit_sql.TIMEID, 5, 2))
transaction_sum_by_month = credit_sql.groupby("month").agg(sum("transaction_value").alias("total_transaction_value"))
sorted_data = transaction_sum_by_month.sort("total_transaction_value", ascending=False)
top_three_months = sorted_data.head(3)
top_three_months_data = [(row.month, row.total_transaction_value) for row in top_three_months]
top_three_months_pd = pd.DataFrame(top_three_months_data, columns=["month", "total_transaction_value"])
top_three_months_pd.plot(kind="bar", x="month", y="total_transaction_value", legend=False)

plt.title("Months with Largest Transaction Data")
plt.xlabel("Month")
plt.ylabel("Transaction Value")
plt.tight_layout()

plt.show()

In [ ]:
credit_sql = credit_sql.withColumn("month", substring(credit_sql.TIMEID, 5, 2))
healthcare_transactions = credit_sql.filter(credit_sql.TRANSACTION_TYPE.like("Healthcare"))
branch_healthcare_total = healthcare_transactions.groupby("BRANCH_CODE").agg(sum("TRANSACTION_VALUE").alias("total_transaction_value"))
top_5_branches = branch_healthcare_total.orderBy(("total_transaction_value")).head(5)
top_5_df = pd.DataFrame(top_5_branches, columns=["BRANCH_CODE", "total_transaction_value"])
top_5_df.plot(kind="bar", x="BRANCH_CODE", y="total_transaction_value", legend=False)

plt.title("Highest Earning Healthcare Transaction Branches")
plt.xlabel("Branch Code")
plt.ylabel("Total Transaction Value")
plt.tight_layout()

plt.show()